In [16]:
import numpy as np
import pandas as pd

import gurobipy as gp
from gurobipy import GRB

# tested with Python 3.7.0 & Gurobi 9.0

In [17]:
# Parametros

g = 5

# Costos

a = [3, 4.05, 4.05, 3.99, 3.88]
b = [20, 18.07, 15.55, 19.21, 26.18]
c = [100, 98.87, 104,26, 107.21, 95.31]

# Emisiones

d = [2, 3.82, 5.01, 1.10, 3.55]
e = [-5, -4.24, -2.15, -3.99, -6.88]
f = [3, 6.09, 5.69, 6.20, 5.57]

# Potencias

Pmin = [28, 90, 68, 76, 19]
Pmax = [206, 284, 189, 266, 53]


In [18]:
model = gp.Model('Generación Térmica ambiental')
# Variables de desición
potgen = model.addVars(g, vtype=GRB.CONTINUOUS, name="potgen")
emigen = model.addVars(g, vtype=GRB.CONTINUOUS, name="emigen")

In [19]:
# Restricción de potencias

p_min = model.addConstrs((potgen[type] >= Pmin[type])
                              for type in range(g))

p_max = model.addConstrs((potgen[type] <= Pmax[type])
                              for type in range(g))

In [20]:
# Satisfacer la demanda

demanda = model.addConstr(gp.quicksum(potgen[type]  for type in range(g)) >= 400)

In [21]:
#Restricción de emisiones

r_emisones = model.addConstr(gp.quicksum(d[type]*potgen[type]*potgen[type]+e[type]*potgen[type]+f[type]
                   for type in range(g)) <= 90000)

In [22]:
# Minimizar costos

costos = gp.quicksum(a[type]*potgen[type]*potgen[type]+b[type]*potgen[type]+c[type]
                    for type in range(g))

model.setObjective(costos)

In [23]:
model.write('cgk3_2_ELI.lp')
model.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 11 rows, 10 columns and 15 nonzeros
Model fingerprint: 0xfc217e8e
Model has 5 quadratic objective terms
Model has 1 quadratic constraint
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 5e+00]
  QLMatrix range   [2e+00, 7e+00]
  Objective range  [2e+01, 3e+01]
  QObjective range [6e+00, 8e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 4e+02]
  QRHS range       [9e+04, 9e+04]
Presolve removed 10 rows and 5 columns
Presolve time: 0.01s
Presolved: 19 rows, 14 columns, 35 nonzeros
Presolved model has 2 second-order cone constraints
Ordering time: 0.00s

Barrier statistics:
 Dense cols : 1
 AA' NZ     : 8.000e+01
 Factor NZ  : 1.920e+02
 Factor Ops : 2.474e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal       

In [24]:
rows = ["Potencia" + str(t) for t in range(g)]
units = pd.DataFrame(columns=range(1), index=rows, data=0.0)

for t in range(g):
            units.loc["Potencia"+str(t)] = potgen[t].x
units

,0
Potencia0,94.212042
Potencia1,90.000170
Potencia2,68.000388
Potencia3,94.788677
Potencia4,52.998781


In [25]:
rows = ["Emisiones" + str(t) for t in range(g)]
units = pd.DataFrame(columns=range(1), index=rows, data=0.0)

for t in range(g):
            units.loc["Emisiones"+str(t)] = d[t]*potgen[t].x*potgen[t].x+e[t]*potgen[t].x+f[t]
units





,0
Emisiones0,17283.757641
Emisiones1,30566.605855
Emisiones2,23025.993703
Emisiones3,9511.375713
Emisiones4,9612.429707
